In [0]:
!pip install dfply
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
from dateutil import parser
import re
import math
from dfply import *
from dateutil.relativedelta import *

In [0]:
# trophies_path = "./trophies.csv"
# trophies = pd.read_csv(trophies_path, index_col = 0)

# mval_path = "/content/drive/My Drive/Capstone/Capstone_Data/full_data/markval.csv"
# markval = pd.read_csv(mval_path, index_col = 0)

# players_path = "/content/drive/My Drive/Capstone/Capstone_Data/full_data/Players_Full.csv"
# players = pd.read_csv(players_path, index_col = 0)


def remove_chars(string):
  return string.strip()

def clean_money(string):
  ammount = string
  try:
    if("free" in string.lower()):
      ammount = 0.0
    elif("k" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000
    elif("m" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000000
  except Exception as e:
    pass
  return ammount

def is_loan(string):
  result = False
  try:
    result = "loan" in string.lower()
  except:
    pass
  return result

def no_loan(string):
  result = False
  try:
    result = "loan" in string.lower()
  except:
    pass
  return not(result)

def clean_loans(string):
  ammount = 0.0
  try:
    if("k" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000
    elif("m" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000000
  except Exception as e:
    pass
  return ammount

def call_nan(string):
  return math.isnan(string)

def get_season(string):
    season = string
    try:
        season = (re.sub('[^\d\/]', '', string))
    except:
        pass
    return season

def clean_height(string):
    height = string
    try:
        height = (re.sub('[^\d\,]', '', string))
        height = height.replace("," , "")
    except:
        pass
    return height

def clean_position(string):
    position = string
    try:
        position = string.split(",")[0]
        position = position.strip()
    except:
        pass
    return position

def positions_list(string):
    positions = string
    try:
        positions = list(string.split(","))
        positions = list(map(lambda x: x.strip(), positions))
    except:
        pass
    return positions

def check_positions(positions, values):
    result = any(np.in1d(positions, values))
    return result

def filter_by_position(players, position, position_attributes):
    pos_bool = players.player_positions.apply(lambda x: check_positions(x,position))
    position_players = players[pos_bool]
    all_positions = list(map(lambda x: x.lower(), position))
    if (position[0] == 'GK'):
        all_positions = []
    pos_attributes = position_attributes + all_positions
    position_players = position_players[pos_attributes]
    return position_players

def get_position_value(string):
    result = string
    try:
        result = string.split("+")[0].strip()
        result = float(result)
    except Exception as e:
        pass
    return result

def season_to_date(season):
    date = season
    try:
      end = season.split("/")[1]
      date = "20" + str(end)
    except:
      pass
    return date

def add_months(date):
    result = date
    try:
       result = date + relativedelta(months=+5) + dt.timedelta(days=+29)
    except:
        pass
    return result

In [0]:
def clean_transfers(transfers):
  transfers = transfers.drop_duplicates()
  transfers["Name"] = transfers["Name"].apply(remove_chars)
  transfers["From"] = transfers["From"].apply(remove_chars)
  transfers["To"] = transfers["To"].apply(remove_chars)
  transfers = transfers[transfers["Player_Id"] != "\nAtalanta "]
  transfers.Player_Id = transfers.Player_Id.astype(int)
  transfers.tm_Id = transfers.tm_Id.astype(int)

  loans = transfers[transfers["Fee"].apply(is_loan)]
  transfers = transfers[(transfers["Fee"].apply(no_loan))]
  transfers["Market Value"] = transfers["Market Value"].apply(clean_money)
  transfers["Market Value"] = pd.to_numeric(transfers["Market Value"], errors="coerce")
  transfers["Fee"] = transfers["Fee"].apply(clean_money)
  transfers["Fee"] = pd.to_numeric(transfers["Fee"], errors="coerce")
  transfers["Loan"] = False

  loans["Market Value"] = loans["Market Value"].apply(clean_money)
  loans["Market Value"] = pd.to_numeric(loans["Market Value"], errors="coerce")
  loans["Fee"] = loans["Fee"].apply(clean_money)
  loans["Fee"] = pd.to_numeric(loans["Fee"], errors="coerce")
  loans["Loan"] = True
  loans.fillna(value = 0, inplace=True)

  transfers = transfers.append(loans)
  transfers["Season"] = transfers["Season"].astype('category')

  transfers.columns = ['Player_Id', 'tm_Id', 'Name', 'From', 'To', 'Fee', 'MV',
       'Season', 'Date', 'Loan']

  transfers = (transfers >>
             arrange(X.Player_Id) >>
             mask(X.Fee.notna()) >>
             drop(X.Player_Id))
  transfers.Season = transfers.Season.astype("category")
  transfers["Date"] = pd.to_datetime(transfers["Date"], errors="coerce")

  # hdf_path = "/content/drive/My Drive/Capstone/Capstone_Data/clean_data/data.h5"
  # transfers.to_hdf(hdf_path, key = "transfers", mode='w', format = "table")
  transfers.to_csv("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/Transfers_Clean.csv")
  transfers.to_pickle("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/Transfers_Clean.pkl")


In [0]:
def clean_markval(markval):
  unique_ids = list(players.tm_Id.unique())
  markval = markval[markval.tm_Id.isin(unique_ids)]
  markval.columns = ["Name", "Club", "League", "Season", "MV", "tm_Id"]
  markval = ( markval >>
              mask(X.MV.notna()) >>
              drop(X.Name) 
              )
  markval = ( markval >>
            select(X.tm_Id, X.Club, X.League, X.Season, X.MV))
  markval[markval.tm_Id == 17259]
  markval["Club"] = markval["Club"].apply(remove_chars)
  markval["League"] = markval["League"].apply(remove_chars)
  markval["MV"] = markval["MV"].apply(clean_money)
  markval["MV"] = pd.to_numeric(markval["MV"], errors="coerce")
  markval.Season = markval.Season.astype("category")
  markval.to_csv("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/markval_Clean.csv")
  markval.to_pickle("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/markval_Clean.pkl")


In [0]:
def clean_trophies(trophies):
    trophies.Season = trophies.Season.apply(get_season)
    trophies.Season = trophies.Season.astype("category")
    trophies.sofifa_id = trophies.sofifa_id.astype("int")
    trophies.Name = trophies.Name.apply(remove_chars)
    trophies.Competition = trophies.Competition.apply(remove_chars)
    trophies.Trophy = trophies.Trophy.astype("category")
    trophies.to_csv("trophies_Clean.csv")
    trophies.to_pickle("trophies_Clean.pkl")

In [0]:
def clean_players(players):
    players.Height = players.Height.apply(clean_height)
    players.Height = pd.to_numeric(players.Height, errors = "coerce")
    players.Followers = pd.to_numeric(players.Followers, errors = "coerce")
    players["Date of Birth"] = pd.to_datetime(players["Date of Birth"], errors = "coerce")
    players["Contract Expires"] = pd.to_datetime(players["Contract Expires"], errors = "coerce")
    players.Team = players.Team.apply(remove_chars)
    players.Nationality = players.Nationality.apply(remove_chars)
    players.Name = players.Name.apply(remove_chars)
    players["Strong Foot"] = players["Strong Foot"].astype("category")
    players["Position"] = players["Position"].astype("category")
    players.columns = ['Id', 'tm_Id', 'Name', 'Team', 'Nationality', 'DOB', 'Height',
       'SF', 'Position', 'Joined', 'Contract_Expires', 'Followers']
    players = ( players >>
          arrange(X.Id) >>
          drop(X.Id))
    players.to_csv("Players_Clean.csv")
    players.to_pickle("Players_Clean.pkl")

In [0]:
def clean_sidelined(sidelined):
    datetimeFormat = '%d/%m/%y'
    sidelined = pd.read_csv("sidelined.csv", index_col = 0)
    sidelined.columns = [c.replace(' ', '_') for c in sidelined.columns]
    sidelined['Start_Date'] = pd.to_datetime(sidelined['Start_Date'], format = datetimeFormat)
    sidelined['End_Date'] = pd.to_datetime(sidelined['End_Date'], format = datetimeFormat)
    sidelined['Duration'] = sidelined['End_Date'] - sidelined['Start_Date']
    sidelined['Duration'] = sidelined['Duration'] / np.timedelta64(1, 'D')
    sidelined['Active'] = np.where(sidelined['End_Date'].isna(), True, False)
    sidelined = ( sidelined >>
                  drop(X.Name))
    sidelined.to_csv("sidelined_Clean.csv")
    sidelined.to_pickle("sidelined_Clean.pkl")


In [0]:
def clean_sofifa(players_20, year):
    players_20["main_position"] = players_20.player_positions.apply(clean_position)
    players_20.player_positions = players_20.player_positions.apply(positions_list)
    players_20.main_position = players_20.main_position.astype('category')
    players_20.team_position = players_20.team_position.astype('category')
    players_20.preferred_foot = players_20.preferred_foot.astype('category')
    players_20.joined = pd.to_datetime(players_20.joined)
    players_20.contract_valid_until = pd.to_datetime(players_20.contract_valid_until, format='%Y', errors = 'coerce') 
    players_20.dob = pd.to_datetime(players_20.dob)
    players_20.work_rate = players_20.work_rate.astype('category')
    players_20["game_year"] = year
    players_20.game_year = pd.to_datetime(players_20.game_year, format='%Y', errors = 'coerce')
    players_20.contract_valid_until = players_20.contract_valid_until.apply(add_months)
    all_positions = ["ls","st","rs","lw","lf","cf","rf","rw","lam",
                 "cam","ram","lm","lcm","cm","rcm","rm","lwb",
                 "ldm","cdm","rdm","rwb","lb","lcb","cb","rcb","rb"]
    players_20[all_positions] = players_20[all_positions].apply(lambda x: x.apply(get_position_value) if x.name in all_positions else x)
    return players_20

def get_head_info(players_20):
    players_head = ( players_20 >> 
                   select(X.sofifa_id, X.short_name, X.long_name, X.age, X.dob, X.height_cm, 
                          X.weight_kg, X.nationality, X.club, X.main_position, X.player_positions,
                          X.overall, X.potential, X.preferred_foot, X.weak_foot, X.skill_moves,
                          X.work_rate,X.team_position,X.joined, X.contract_valid_until, X.game_year ))
    return players_head

In [0]:
def get_teams_leagues(markval):
    markval["Club"] = markval["Club"].apply(remove_chars)
    markval["League"] = markval["League"].apply(remove_chars)
    markval["Date"] = markval.Season.apply(season_to_date)
    markval["Date"] = pd.to_datetime(markval.Date,format='%Y', errors = 'coerce')
    markval["Date"] = markval["Date"].apply(add_months)
    markval["Date"] = pd.to_datetime(markval.Date,format='%YYYY-%MM-%DD', errors = 'coerce')
    teams_leagues = ( markval >> 
        group_by(X.Club, X.League, X.Season, X.Date) >>
        summarise(Club = X.Club.unique()[0], League = X.League.unique()[0], 
        Season = X.Season.unique()[0], Date = X.Date.unique()[0])
    )
    teams_leagues.set_index("Date", drop=True, inplace=True)
    return teams_leagues

In [0]:
def append_or_create(path, df):
    try:
        df1 = pd.read_pickle(path)
        df2 = df1.append(df)
        df2.to_pickle(path)
    except:
        df.to_pickle(path)

def get_all_positions(players_20):
    strikers_20 = filter_by_position(players_20, strikers, striker_attributes)
    wingers_20 = filter_by_position(players_20, wingers, winger_attributes)
    def_mids_20 = filter_by_position(players_20, def_midfielders, def_midfielders_attributes)
    clean_mids_20 = filter_by_position(players_20, clean_midfielders, clean_midfielders_attributes)
    att_mids_20 = filter_by_position(players_20, att_midfielders, att_midfielders_attributes)
    full_backs_20 = filter_by_position(players_20, full_backs, full_backs_attributes)
    defenders_20 = filter_by_position(players_20, defenders, defenders_attributes)
    goalkeepers_20 = filter_by_position(players_20, goalkeepers, goalkeepers_attributes)
    id_vars = ['sofifa_id', 'player_positions', 'main_position',
       'team_position', 'game_year']
    strikers_20_long = pd.melt(strikers_20, id_vars = id_vars, var_name = "Attribute")
    wingers_20_long = pd.melt(wingers_20, id_vars = id_vars, var_name = "Attribute")
    def_mids_20_long = pd.melt(def_mids_20, id_vars = id_vars, var_name = "Attribute")
    clean_mids_20_long = pd.melt(clean_mids_20, id_vars = id_vars, var_name = "Attribute")
    att_mids_20_long = pd.melt(att_mids_20, id_vars = id_vars, var_name = "Attribute")
    full_backs_20_long = pd.melt(full_backs_20, id_vars = id_vars, var_name = "Attribute")
    defenders_20_long = pd.melt(defenders_20, id_vars = id_vars, var_name = "Attribute")
    goalkeepers_long = pd.melt(goalkeepers_20, id_vars = id_vars, var_name = "Attribute")
    
    all_positions_long = pd.concat([strikers_20_long, wingers_20_long, def_mids_20_long, clean_mids_20_long, att_mids_20_long, full_backs_20_long,
                                   defenders_20_long, goalkeepers_long], sort=False)
    
    append_or_create("positions/strikers_Clean.pkl",strikers_20)
    append_or_create("positions/wingers_Clean.pkl",wingers_20)
    append_or_create("positions/def_mids_Clean.pkl",def_mids_20)
    append_or_create("positions/clean_mids_Clean.pkl",clean_mids_20)
    append_or_create("positions/full_backs_Clean.pkl",full_backs_20)
    append_or_create("positions/defenders_Clean.pkl",defenders_20)
    append_or_create("positions/goalkeepers_Clean.pkl",goalkeepers_20)
    append_or_create("sofifa_long_Clean.pkl",all_positions_long)

In [0]:
# players_20.head(20)


strikers = ['CF', 'ST', 'LS', 'RS']
wingers = ['LW', 'RW', 'LF', 'RF']
clean_midfielders = ['CM', 'LCM', 'RCM','LM', 'RM']
def_midfielders = ['CDM','RDM', 'LDM']
att_midfielders = ['CAM', 'RAM', 'LAM']
full_backs = ['LB', 'RB', 'LWB', 'RWB']
defenders = ['CB', 'LCB', 'RCB']
goalkeepers = ['GK']
striker_attributes = ['sofifa_id', 'player_positions', 'main_position', 'team_position','game_year',
                      'pace', 'shooting', 'passing', 'physic', 'dribbling', 
                      'attacking_crossing','attacking_finishing', 
                      'attacking_heading_accuracy', 'attacking_short_passing',
                    'attacking_volleys', 'movement_acceleration', 
                      'movement_sprint_speed', 'movement_agility',
                      'power_stamina', 'mentality_positioning']
clean_midfielders_attributes = ['sofifa_id', 'player_positions', 'main_position', 'team_position',
                                'game_year',
                                 'passing', 'dribbling', 'attacking_short_passing',
                                'skill_ball_control', 'power_stamina', 'mentality_vision'
                               ]
winger_attributes = ['sofifa_id', 'player_positions', 'main_position', 'team_position','game_year',
                     'pace', 'passing', 'dribbling', 'attacking_crossing', 
                     'attacking_short_passing', 'movement_acceleration','movement_sprint_speed', 
                     'movement_agility', 'power_stamina', 'mentality_vision'] #skill_moves
def_midfielders_attributes = ['sofifa_id',  'player_positions','main_position', 'team_position',
                              'game_year',
                            'defending', 'mentality_aggression', 'mentality_interceptions', 'defending_marking', 
                              'defending_standing_tackle','defending_sliding_tackle']
att_midfielders_attributes = ['sofifa_id', 'player_positions', 'main_position', 'game_year',
                              'team_position','pace', 'passing', 'attacking_short_passing', 'skill_ball_control', 
                              'power_stamina', 'power_long_shots', 'mentality_vision']
defenders_attributes = ['sofifa_id','player_positions','main_position','team_position','game_year',
                        'physic', 'defending', 'mentality_aggression', 
                        'mentality_interceptions','defending_marking', 'defending_standing_tackle',
                        'defending_sliding_tackle', 'attacking_heading_accuracy','power_jumping', 
                        'mentality_composure']
full_backs_attributes = ['sofifa_id','player_positions','main_position','team_position', 'game_year',
                         'pace', 'defending', 'mentality_aggression', 'attacking_crossing',
                        'mentality_interceptions','defending_marking', 'defending_standing_tackle',
                        'defending_sliding_tackle','mentality_composure']
goalkeepers_attributes = ['sofifa_id','player_positions', 'main_position','team_position', 'game_year',
                          'gk_diving','gk_handling','gk_kicking','gk_reflexes',
                          'gk_speed','gk_positioning']
all_positions = ["ls","st","rs","lw","lf","cf","rf","rw","lam",
             "cam","ram","lm","lcm","cm","rcm","rm","lwb",
             "ldm","cdm","rdm","rwb","lb","lcb","cb","rcb","rb"]

# player_attributes = []
players_20 = pd.read_csv("players_15.csv")
players_20 = clean_sofifa(players_20, year = 2015)
players_head = get_head_info(players_20)
# players_20
append_or_create("sofifa_head_Clean.pkl", players_head)
pd.read_pickle("sofifa_head_Clean.pkl")

,sofifa_id,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,main_position,...,overall,potential,preferred_foot,weak_foot,skill_moves,work_rate,team_position,joined,contract_valid_until,game_year
0,158023,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,RW,...,94,94,Left,4,4,Medium/Low,RW,2004-07-01,2021-06-30,2020-01-01
1,20801,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,ST,...,93,93,Right,4,5,High/Low,LW,2018-07-10,2022-06-30,2020-01-01
2,190871,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,LW,...,92,92,Right,5,5,High/Medium,CAM,2017-08-03,2022-06-30,2020-01-01
3,200389,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,GK,...,91,93,Right,3,1,Medium/Medium,GK,2014-07-16,2023-06-30,2020-01-01
4,183277,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,LW,...,91,91,Right,4,4,High/Medium,LW,2019-07-01,2024-06-30,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15460,220806,E. Redman,Ellis Redman,17,1996-09-26,187,81,Wales,Newport County,CB,...,41,61,Right,2,2,Medium/Medium,RES,2013-10-22,2015-06-30,2015-01-01
15461,225509,A. Collins,Aaron Collins,17,1997-06-01,178,57,Wales,Newport County,ST,...,41,50,Right,3,3,Medium/Medium,RES,2014-08-14,2015-06-30,2015-01-01
15462,217591,P. Żemło,Piotr Żemło,18,1995-07-10,190,76,Poland,Wisła Kraków,LM,...,40,50,Left,3,2,Medium/Medium,RES,2012-07-01,2015-06-30,2015-01-01
15463,222947,T. Davies,Tom Davies,22,1992-04-18,180,70,England,Fleetwood Town,CB,...,40,49,Right,2,2,Medium/Medium,RES,2014-03-11,2021-06-30,2015-01-01


In [0]:
# get_all_positions(players_20)

In [0]:
# append_or_create("sofifa_head_Clean.pkl", players_head)

In [0]:
# markval = pd.read_csv("markval.csv")
# teams_leagues = get_teams_leagues(markval)
# teams_leagues

,Club,League,Season
Date,,,
2006-06-30,1. FC Köln,Bundesliga,05/06
2007-06-30,1. FC Köln,Bundesliga,06/07
2008-06-30,1. FC Köln,Bundesliga,07/08
2009-06-30,1. FC Köln,Bundesliga,08/09
2010-06-30,1. FC Köln,Bundesliga,09/10
...,...,...,...
2016-06-30,Östersunds FK,Allsvenskan,15/16
2017-06-30,Östersunds FK,Allsvenskan,16/17
2018-06-30,Östersunds FK,Allsvenskan,17/18


In [0]:
# teams_leagues.to_pickle("teams_leagues_Clean.pkl")